In [2]:
import cv2
import depthai as dai
import numpy as np

import time
from pathlib import Path
from matplotlib import pyplot as plt

import os, random

In [3]:
# Start defining a pipeline
pipeline = dai.Pipeline()

# Define a source - left grayscale cameras
cam_left = pipeline.createMonoCamera()
cam_left.setBoardSocket(dai.CameraBoardSocket.LEFT)
cam_left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)

# Create outputs
xout_left = pipeline.createXLinkOut()
xout_left.setStreamName('left')
cam_left.out.link(xout_left.input)

In [ ]:
## RECORDING FOR 10 SECONDS
## THE FRAMES ARE BEING STORED IN DIRECTORY "video_frames"

# Connect and start the pipeline
with dai.Device(pipeline) as device:

    # Output queue will be used to get the grayscale frames from the output defined above
    q = device.getOutputQueue(name="left", maxSize=4, blocking=False)

    # Make sure the destination path is present before starting to store the examples
    Path(f"../video_frames/").mkdir(parents=True, exist_ok=True)

    # running loop for 10 secs
    ten_secs = time.time() + 10
    
    while time.time() < ten_secs:
        # Blocking call, will wait until a new data has arrived
        inSrc = q.get()  
        # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
        frame = inSrc.getCvFrame()
        # Frame is transformed and ready to be shown
        cv2.imshow("left", frame)
        cv2.waitKey(1)

        cv2.imwrite(f"../video_frames/{int(time.time() * 10000)}.png", frame)

    cv2.destroyAllWindows()            

After recording a 10 seconds video, 
I selected an image and cropped out 
the region of interest from it.


In [4]:
# Displaying the region of interest 

roi = cv2.imread("roi.png")
roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

cv2.imshow('Region of interest', roi)
cv2.waitKey(1000)
cv2.destroyAllWindows()  

## Sum of squared differences

In [5]:
Path(f"pattern_match_ssd").mkdir(parents=True, exist_ok=True)

for i in range(10):
    # loading a random image
    rand_img_file = random.choice(os.listdir("../video_frames/"))
    rand_img = cv2.imread("../video_frames/"+rand_img_file)
    gray = cv2.cvtColor(rand_img, cv2.COLOR_BGR2GRAY)

    # Create Summed of Squared Differences with patch
    ssd = cv2.matchTemplate(gray, roi, cv2.TM_SQDIFF_NORMED)

    # find min for match with SQDIFF_NORMED
    point = np.where(ssd == ssd.min())
    y = point[0][0]
    x = point[1][0]
    w = len(roi[0])
    l = len(roi)

    # Draw Rectangle
    pm = cv2.rectangle(rand_img, (x, y), (x+w, y+l), (255, 0, 0), 2)

    cv2.imshow("Pattern Match", pm) 
    cv2.waitKey(1000)
    cv2.imwrite("pattern_match_ssd/out_"+rand_img_file, pm)
    cv2.destroyAllWindows()  


## Normalized correlation

In [6]:
Path(f"pattern_match_ncor").mkdir(parents=True, exist_ok=True)

for i in range(10):
    # loading a random image
    rand_img_file = random.choice(os.listdir("../video_frames/"))
    rand_img = cv2.imread("../video_frames/"+rand_img_file)
    gray = cv2.cvtColor(rand_img, cv2.COLOR_BGR2GRAY)

    # Create Summed of Squared Differences with patch
    ssd = cv2.matchTemplate(gray, roi, cv2.TM_SQDIFF_NORMED)

    # find min for match with SQDIFF_NORMED
    point = np.where(ssd == ssd.min())
    y = point[0][0]
    x = point[1][0]
    w = len(roi[0])
    l = len(roi)

    # Draw Rectangle
    pm = cv2.rectangle(rand_img, (x, y), (x+w, y+l), (255, 0, 0), 2)

    cv2.imshow("Pattern Match", pm) 
    cv2.waitKey(1000)
    cv2.imwrite("pattern_match_ncor/out_"+rand_img_file, pm)
    cv2.destroyAllWindows()  
